In [1]:
from langgraph.graph import END, START, StateGraph
from typing import TypedDict
import subprocess
from openai import OpenAI

class State(TypedDict):

    video_file: str
    audio_file: str
    transcription: str

In [2]:
def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4","mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file,
    ]
    subprocess.run(command)
    return {"audio_file": output_file}

def transcribe_audio(state: State):
    client = OpenAI()
    with open(state["audio_file"], "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            language="ko",
            prompt="Jacqueline, Isabele"
        )
    return {"transcription": transcript}


In [3]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_edge("transcribe_audio", END)

graph = graph_builder.compile()

In [4]:
graph.invoke({"video_file": "fun.mp4"})

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.6.3.2)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gpl --enable-libsvtav1 --enable-libopus --enable-libx264 --enable-libmp3lame --enable-libdav1d --enable-libvpx --enable-libx265 --enable-openssl --enable-videotoolbox --enable-audiotoolbox --enable-neon
  libavutil      60.  8.100 / 60.  8.100
  libavcodec     62. 11.100 / 62. 11.100
  libavformat    62.  3.100 / 62.  3.100
  libavdevice    62.  1.100 / 62.  1.100
  libavfilter    11.  4.100 / 11.  4.100
  libswscale      9.  1.100 /  9.  1.100
  libswresample   6.  1.100 /  6.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'fun.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    encoder         : Google
  Duration: 00:02:41.49, sta

{'video_file': 'fun.mp4',
 'audio_file': 'fun.mp3',
 'transcription': '자베리는 지금 뉴스 촬영에 푹 빠져있다. 무화지경에 빠져 챌린지 중인 자베리. 보고 있으면 피가 거꾸로 솟는다. 어머나! 나가요! 피가 다 엉췄... 자베라! 정말 진짜 진짜! 쟤 드라이밍이 크레이지! 한편, 귤락을 골라내는 섬세함이다. 영국의 그레이스도 K-드라마에 푹 빠졌다. 아, 정말 이거 다 빨라, 정말로! 진짜 그라이스다, 자베라 정말로 진짜! 어디론가 전화를 거는 그레이스도 여보, 오늘 올 때 마이크랑 그러면 사와. 당연히 레드지, 내가 피는 거! 몰라, 오늘은 그냥 시그널 피해야겠다. 오늘은 피해야 돼, 그냥 사와! 이날은 담배 없이는 버틸 수 없었다. 끊어요. 거의 영국인답게 차는 잊지 않는다. 그래, 이제는 나도 어쩔 수가 없어. 그냥 가, 그래! 불현듯 어디론가 전화를 거는 그레이스 노래를 이은미 창법으로 부른다. 여러분, Who is this? 어, 나야 유난다. 나 그레이스야. 어, 그레이스! 나 지금 오피스에서 스테인리스에서 있었는데 그레이스가 딱 전화를 했어. 유난다, 나 자베리 때문에 그냥 한국 가려고 그냥. 근데 한국을 갔다니? 어, 더 이상 미룰 수가 없을 것 같아. 그때 이제 한국에 에어앱 B&B인가 그거 무슨 사습지 같은 거 끝나고 그러셨지? 어, 맞아요, 언니. 우리 마이크맨 여기 비거리 힐 이제 우리 집 팔고 한국 가가지고 그거는 좀 하고 그랬거든. 어머, 그래? 소문에 린나 언니가 막 한국에서 브랜드처럼 산다고 막 그랬는데. 이제 린나 아니고 제클린이라니까! 이제 잘 있으니까 이상한 소리 좀 그만하고 다녀, 너! 제클린 험담에 속상해진 그레이스 쉿! 그레이스야! 뭐야? 다시 시작된 유난다의 이은미 창법 어디론가 또다시 전화를 거는 그레이스\n'}